In [55]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# from dataprep.eda import plot, plot_correlation, plot_missing

import plotly 
import plotly.express as px
# from plotly import tools, subplots
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# from plotly.offline import init_notebook_mode, iplot
# import plotly.graph_objs as go
# import plotly.express as px

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


In [56]:
test = pd.read_csv("/Users/seungji/Desktop/Dacon/Daesamanlap/ProcessedData/merged_test.csv")
train = pd.read_csv("/Users/seungji/Desktop/Dacon/Daesamanlap/ProcessedData/merged_train.csv")

In [57]:
tr_gr = train.groupby("단지코드").mean().reset_index()
tst_gr = test.groupby("단지코드").mean().reset_index()
지역 = []
for d in train.단지코드.unique():
    지역.append(train.loc[train.단지코드 == d,"지역"].unique()[0])
tr_gr["지역"] = 지역
# train["지역구분"] = [0]*len(train)
tr_gr.loc[tr_gr.지역.isin(['경상남도', '경기도', '전라북도', '강원도',  '충청남도','제주특별자치도',  '충청북도', '전라남도', '경상북도', '세종특별자치시']),"지역"] = "8도"
tr_gr.loc[tr_gr.지역.isin(['대전광역시', '광주광역시', '울산광역시', '대구광역시', '서울특별시','부산광역시']),"지역"] = "광역시"

# 자격유형 = []
# for d in train.단지코드.unique():
#     자격유형.append(train.loc[train.단지코드 == d,"자격유형"].unique()[0])
# tr_gr["자격유형"] = 자격유형

# 공급유형 = []
# for d in train.단지코드.unique():
#     공급유형.append(train.loc[train.단지코드 == d,"공급유형"].unique()[0])
# tr_gr["공급유형"] = 공급유형

tr_gr["전용면적_std"] = train.groupby("단지코드").std().reset_index()["전용면적"]
tr_gr["전용면적별세대수_std"] = train.groupby("단지코드").std().reset_index()["전용면적별세대수"]
tr_gr["임대보증금_std"] = train.groupby("단지코드").std().reset_index()["임대보증금"]
tr_gr["임대료_std"] = train.groupby("단지코드").std().reset_index()["임대료"]
tst_gr["전용면적_std"] = test.groupby("단지코드").std().reset_index()["전용면적"]
tst_gr["전용면적별세대수_std"] = test.groupby("단지코드").std().reset_index()["전용면적별세대수"]
tst_gr["임대보증금_std"] = test.groupby("단지코드").std().reset_index()["임대보증금"]
tst_gr["임대료_std"] = test.groupby("단지코드").std().reset_index()["임대료"]
tr_re = pd.concat([train.단지코드,pd.get_dummies(train["지역"])],axis=1).groupby("단지코드").sum()
tst_re = pd.concat([test.단지코드,pd.get_dummies(test["지역"])],axis=1).groupby("단지코드").sum()
tr_re[tr_re>0] = 1
tst_re[tst_re>0] = 1
tr_gr = tr_gr.merge(tr_re, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_re, left_on = "단지코드",right_on ="단지코드",how = "left")
tr_apart = pd.concat([train.단지코드,pd.get_dummies(train["임대건물구분"])],axis=1).groupby("단지코드").sum().reset_index()
tst_apart = pd.concat([test.단지코드,pd.get_dummies(test["임대건물구분"])],axis=1).groupby("단지코드").sum().reset_index()
tr_gr = tr_gr.merge(tr_apart, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_apart, left_on = "단지코드",right_on ="단지코드",how = "left")
tr_gong = pd.concat([train.단지코드, pd.get_dummies(train["공급유형"])],axis =1)
tst_gong = pd.concat([test.단지코드, pd.get_dummies(test["공급유형"])],axis =1)
tr_gong["공공임대(5년,10년)"] = tr_gong["공공임대(5년)"]+tr_gong["공공임대(10년)"]
tst_gong["공공임대(5년,10년)"] = tr_gong["공공임대(10년)"]
tr_gong = tr_gong[['단지코드',   '공공임대(50년)',  '공공임대(분납)',
       '국민임대', '영구임대', '임대상가',  '행복주택', '공공임대(5년,10년)']]
tst_gong = tst_gong[['단지코드',   '공공임대(50년)',  '공공임대(분납)',
       '국민임대', '영구임대', '임대상가',  '행복주택', '공공임대(5년,10년)']]
tr_gong = tr_gong.groupby("단지코드").sum()
tst_gong = tst_gong.groupby("단지코드").sum()
tr_gong[tr_gong>0] = 1
tst_gong[tst_gong>0] = 1
tr_gong = tr_gong.reset_index()
tst_gong = tst_gong.reset_index()

tr_gr = tr_gr.merge(tr_gong, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_gong, left_on = "단지코드",right_on ="단지코드",how = "left")

tr_ja = pd.concat([train.단지코드,pd.get_dummies(train["자격유형"])],axis=1).groupby("단지코드").sum()
tst_ja = pd.concat([test.단지코드,pd.get_dummies(test["자격유형"])],axis=1).groupby("단지코드").sum()

tr_ja[tr_ja>0] = 1
tst_ja[tst_ja>0] = 1

tr_ja = tr_ja[tst_ja.columns]
tr_ja = tr_ja.reset_index()
tst_ja = tst_ja.reset_index()

tr_gr = tr_gr.merge(tr_ja, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_ja, left_on = "단지코드",right_on ="단지코드",how = "left")
tr_gr =tr_gr.drop("서울특별시",axis=1)

tr_gr.loc[tr_gr.전용면적_std.isna(),"전용면적_std"] = 0
tr_gr.loc[tr_gr.전용면적별세대수_std.isna(),"전용면적별세대수_std"] = 0
tr_gr.loc[tr_gr.임대보증금_std.isna(),"임대보증금_std"] = 0
tr_gr.loc[tr_gr.임대료_std.isna(),"임대료_std"] = 0

In [58]:
train = tr_gr

In [59]:
train["등록차량수/총세대수"] = train["등록차량수"]/train["총세대수"]
train["등록차량수/연면적"] = train["등록차량수"]/train["연면적"]
train["임대보증금/임대료"] = train["임대보증금"]/train["임대료"]

train["지역"] = [0]*len(train)
train.loc[train.지역.isin(['경상남도', '경기도', '전라북도', '강원도',  '충청남도','제주특별자치도',  '충청북도', '전라남도', '경상북도', '세종특별자치시']),"지역"] = 0
train.loc[train.지역.isin(['대전광역시', '광주광역시', '울산광역시', '대구광역시', '서울특별시','부산광역시']),"지역"] = 1


In [60]:
train.columns

Index(['단지코드', '총세대수', '전용면적', '전용면적별세대수', '공가수', '임대보증금', '임대료', '지하철역',
       '버스정류장', '단지내주차면수', '등록차량수', '연면적', '위도', '경도', 'subway_dist', '환승역 수',
       '총인구수', '세대당_인구', '남/여비율', '남/여_0~19세', '남/여_20~39세', '남/여_40~69세',
       '남/여_70세이상', '0~19세_비율', '20~39세_비율', '40~69세_비율', '70세이상_비율', '지역',
       '전용면적_std', '전용면적별세대수_std', '임대보증금_std', '임대료_std', '강원도', '경기도',
       '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '세종특별자치시', '울산광역시',
       '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도', '상가', '아파트', '공공임대(50년)',
       '공공임대(분납)', '국민임대', '영구임대', '임대상가', '행복주택', '공공임대(5년,10년)', 'A', 'C',
       'D', 'E', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', '등록차량수/총세대수',
       '등록차량수/연면적', '임대보증금/임대료'],
      dtype='object')

In [61]:
tr

NameError: name 'tr' is not defined

In [62]:
train = train.drop(["단지코드"],axis =1)

In [63]:
from sklearn.preprocessing import StandardScaler
x = StandardScaler().fit_transform(train)

In [64]:
from sklearn.decomposition import PCA

In [75]:
pca = PCA(n_components=15)
pc = pca.fit_transform(x)
principalDf = pd.DataFrame(data = pc)

In [80]:
train.columns

Index(['총세대수', '전용면적', '전용면적별세대수', '공가수', '임대보증금', '임대료', '지하철역', '버스정류장',
       '단지내주차면수', '등록차량수', '연면적', '위도', '경도', 'subway_dist', '환승역 수', '총인구수',
       '세대당_인구', '남/여비율', '남/여_0~19세', '남/여_20~39세', '남/여_40~69세', '남/여_70세이상',
       '0~19세_비율', '20~39세_비율', '40~69세_비율', '70세이상_비율', '지역', '전용면적_std',
       '전용면적별세대수_std', '임대보증금_std', '임대료_std', '강원도', '경기도', '경상남도', '경상북도',
       '광주광역시', '대구광역시', '대전광역시', '부산광역시', '세종특별자치시', '울산광역시', '전라남도', '전라북도',
       '제주특별자치도', '충청남도', '충청북도', '상가', '아파트', '공공임대(50년)', '공공임대(분납)', '국민임대',
       '영구임대', '임대상가', '행복주택', '공공임대(5년,10년)', 'A', 'C', 'D', 'E', 'G', 'H',
       'I', 'J', 'K', 'L', 'M', 'N', '등록차량수/총세대수', '등록차량수/연면적', '임대보증금/임대료'],
      dtype='object')

In [82]:
pd.concat([pd.DataFrame(train.columns),pd.DataFrame(pca.components_).transpose()],axis = 1)

,0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,총세대수,-0.0514,0.2854,-0.0530,0.0534,0.2274,-0.1285,0.0971,0.0502,0.1645,0.0634,0.0162,-0.0531,-0.0217,-0.1099,-0.1558
1,전용면적,-0.0858,0.0300,-0.3518,0.1031,-0.1301,0.2307,0.0222,-0.0008,-0.0267,0.0918,-0.0256,0.0690,-0.0124,0.0318,0.0589
2,전용면적별세대수,-0.1113,0.0575,-0.1071,0.1551,0.1225,-0.2090,-0.0496,0.0935,0.0291,-0.1206,0.3167,0.0771,-0.0122,-0.1968,0.0694
3,공가수,-0.0978,0.0435,0.1718,0.2298,0.2382,-0.0738,-0.0002,0.1293,-0.0346,0.0750,0.0752,0.0395,0.0737,-0.1003,-0.0172
4,임대보증금,-0.1751,0.0864,0.0217,-0.0343,-0.0028,0.3975,0.1007,0.0986,-0.0831,-0.0368,-0.0327,0.0648,0.0247,-0.0248,0.0637
5,임대료,0.1490,0.1703,-0.0976,0.0524,-0.0073,0.0564,0.0196,-0.0241,0.0771,0.1759,-0.1306,0.2466,-0.1092,-0.0968,-0.0345
6,지하철역,0.0224,0.0860,-0.0491,-0.1026,0.0791,0.0396,-0.1290,0.0575,0.0817,-0.3751,0.0273,0.0873,0.1419,0.2931,-0.2226
7,버스정류장,-0.0405,0.0817,0.0040,-0.0173,-0.0354,-0.0266,0.0442,-0.0555,-0.1761,-0.0219,0.1314,0.1329,0.2137,0.0406,-0.1072
8,단지내주차면수,-0.2209,0.1666,-0.0825,0.0747,0.1828,-0.0844,0.1145,0.0466,0.1306,0.0362,-0.0141,-0.0910,-0.0127,-0.0969,-0.1331
9,등록차량수,-0.2260,0.1265,-0.1591,0.0679,0.0810,0.0108,0.1125,0.0032,0.1523,0.0670,-0.0177,-0.1299,-0.0622,-0.0660,-0.0945


In [77]:
pca.explained_variance_

array([8.9540767 , 6.34764302, 3.90513222, 3.15543309, 2.99610643,
       2.88889849, 2.30313783, 2.14435389, 2.04427581, 1.78085094,
       1.74314955, 1.61434141, 1.58209067, 1.48092756, 1.463984  ])

In [88]:
# pca1 = PCA(n_components=1)
# X_low = pca1.fit_transform(X)
X2 = pca.inverse_transform(pc)

plt.figure(figsize=(7, 7))
ax = sns.scatterplot(0, 1, data=pd.DataFrame(x), s=100, color=".2", marker="s")
for i in range(len(train)):
    d = 0.03 if x[i, 1] > X2[i, 1] else -0.04
    ax.text(x[i, 0] - 0.065, x[i, 1] + d, "표본 {}".format(i + 1))
    plt.plot([x[i, 0], X2[i, 0]], [x[i, 1], X2[i, 1]], "k--")
plt.plot(X2[:, 0], X2[:, 1], "o-", markersize=10)
plt.plot(x[:, 0].mean(), x[:, 1].mean(), markersize=10, marker="D")
plt.axvline(x[:, 0].mean(), c='r')
plt.axhline(x[:, 1].mean(), c='r')
plt.grid(False)
plt.xlabel("꽃받침 길이")
plt.ylabel("꽃받침 폭")
plt.title("Iris 데이터의 1차원 차원축소")
plt.axis("equal")
plt.show()

TypeError: object of type 'int' has no len()

<Figure size 504x504 with 0 Axes>